In [1]:
# Load required modules
import sys
import numpy as np
import xarray as xr
from datetime import date
import datetime as dt
import scipy.io
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy import integrate
import pandas as pd
%pylab inline
# Load marineHeatWaves definition module
import os
sys.path.insert(0,'x:/heat_content/')
import marineHeatWave as marineHeatWave
#import marineHeatWaves as mhw
import pdb

Populating the interactive namespace from numpy and matplotlib


Load all the yearly heat content files for the 31 year historical run

In [2]:
ic=0
ny=np.arange(1980,2020)
for i in ny:
    file='x:/heat_content/heat_wave_data/West_Coast_Temperature_'+str(i)+'_last4days.nc'
    ds=xr.open_dataset(file)
    print(file)
    tz=ds['time']
    varname='temperature_'+str(i)+'_surface'
    #if i==1981:
    #    varname='heat_content_1980_100_meters'
    #if i==1985:
    #    varname='heat_content_1984_100_meters'
    th=ds[varname]
    if ic==0:
        bigarray=th
        bigtime=tz
        ic=1;
    else:
        bigarray=np.dstack((bigarray,th))
        bigtime=np.append(bigtime,tz)

x:/heat_content/heat_wave_data/West_Coast_Temperature_1980_last4days.nc
x:/heat_content/heat_wave_data/West_Coast_Temperature_1981_last4days.nc
x:/heat_content/heat_wave_data/West_Coast_Temperature_1982_last4days.nc
x:/heat_content/heat_wave_data/West_Coast_Temperature_1983_last4days.nc
x:/heat_content/heat_wave_data/West_Coast_Temperature_1984_last4days.nc
x:/heat_content/heat_wave_data/West_Coast_Temperature_1985_last4days.nc
x:/heat_content/heat_wave_data/West_Coast_Temperature_1986_last4days.nc
x:/heat_content/heat_wave_data/West_Coast_Temperature_1987_last4days.nc
x:/heat_content/heat_wave_data/West_Coast_Temperature_1988_last4days.nc
x:/heat_content/heat_wave_data/West_Coast_Temperature_1989_last4days.nc
x:/heat_content/heat_wave_data/West_Coast_Temperature_1990_last4days.nc
x:/heat_content/heat_wave_data/West_Coast_Temperature_1991_last4days.nc
x:/heat_content/heat_wave_data/West_Coast_Temperature_1992_last4days.nc
x:/heat_content/heat_wave_data/West_Coast_Temperature_1993_last4

In [3]:
bigtime[-1]

numpy.datetime64('2019-04-29T12:00:00.000000000')

Load the current heat content run data

In [4]:
file='x:/heat_content/heat_wave_data/West_Coast_Temperature_surface_2_20220809.nc'
ds=xr.open_dataset(file)
tz=ds['time']

In [6]:
ind=np.where(tz > bigtime[-1])

In [7]:
ind

(array([3040, 3041, 3042, ..., 4229, 4230, 4231], dtype=int64),)

In [8]:
sst=ds['temperature_surface'].values
sst=sst.transpose((1,2,0))

In [10]:
newsst=sst[:,:,3040:]
atime=tz[3040:]

In [11]:
#sst=ds['temperature_surface'].values
#sst=sst.transpose((1,2,0))
oldarray=bigarray;
oldtime=bigtime;
#bigarray=np.dstack((bigarray,sst))
bigarray=np.dstack((bigarray,newsst))
#bigtime=np.append(bigtime,tz)
bigtime=np.append(bigtime,atime)

In [ ]:
plt.plot(oldtime,oldarray[50,50,:])
plt.plot(tz,sst[50,50,:],'r')

In [ ]:
plt.savefig('model_sst50x50_20220708_30yrandNRT.png')

In [ ]:
import numpy as np
z=np.arange(1981,2021)

In [ ]:
x=datetime.datetime(1980,1,9,12,0,0)
print(x)

In [ ]:
ts=oldtime-np.datetime64('1970-01-01T00:00:00Z')
#import pandas as pd
#x=pd.Timestamp(oldtime)
#print(x)
#from datetime import datetime
#ii=oldtime[0]
#iz=ii.astype(datetime)
#print(iz)

In [ ]:
xs=ts/np.timedelta64(1,'s')

In [ ]:
datetime.utcfromtimestamp(xs)

In [ ]:
i=np.where(oldtime.asdatetime==x)
print(i)

In [ ]:
oldtime[0]

In [ ]:
print(bigtime.shape)
print(bigarray.shape)

Access the relevant variables

Rearrange the columns to match the 31 year data already loaded so we can merge the data

Check dimensions

Now to actually compute the heat wave information.

In [12]:
#[nt,nlo,nla]=GFNMS_mask['heat_content_100_meters'].shape
[nlo,nla,nt]=bigarray.shape

In [13]:
alat=ds['lat']
alon=ds['lon']

In [14]:
time=bigtime
#time=ds['time'].values
newtime=[]
it=np.arange(0,len(time))
for t in it:    
    tx=dt.datetime.utcfromtimestamp(time[t].tolist()/1e9)
    tx=dt.datetime.toordinal(tx)
    newtime=np.append(newtime,tx)
newtime=newtime.astype('int')

In [16]:
newtime[0]

722823

In [17]:
heatwavesnumber=np.zeros((175,117))
hwv=np.zeros((175,117))
hwevents=np.zeros((175,117))
hwmoderate=np.zeros((175,117))
hwstrong=np.zeros((175,117))
hwsevere=np.zeros((175,117))
hwextreme=np.zeros((175,117))
startindexs=[]
stopindexs=[]
heatvalueindex=np.zeros((nt,nlo,nla))
heatmoderate=np.zeros((nt,nlo,nla))
heatstrong=np.zeros((nt,nlo,nla))
heatsevere=np.zeros((nt,nlo,nla))
heatextreme=np.zeros((nt,nlo,nla))

In [ ]:
#plt.plot(bigtime,bigarray[1,1,:])

In [18]:
ix=np.arange(0,175)
iy=np.arange(0,117)
# first actual data at i=69, j=59
for i in ix:
    print(i)
    for j in iy:
        xtest=bigarray[i,j,:]
        #xtest=test.values        
        # how to check for NaN
        if np.isnan(xtest[0])==False:
            # we have data to run the code on
            # here is where we can run the marine Heat Wave code
            mhw=marineHeatWave.detect(newtime,xtest)
            stats=mhw[0]
            other=mhw[1]
            indexs=stats['index_start']
            indexe=stats['index_end']
            idiff=(np.array(indexe)-np.array(indexs))+1
            heatwavesnumber[i,j]=sum(idiff)
            hwv[i,j]=sum(stats['duration'])
            hwmoderate[i,j]=sum(stats['duration_moderate'])
            hwstrong[i,j]=sum(stats['duration_strong'])
            hwsevere[i,j]=sum(stats['duration_severe'])
            hwextreme[i,j]=sum(stats['duration_extreme'])
            hwevents[i,j]=stats['n_events']
            startindexs.append(mhw[0]['index_start'])
            stopindexs.append(mhw[0]['index_end'])
            # keep track of when a heat wave occurs.  Need to know type of heat wave also?
            category=stats['category']
            lid=len(indexs)
            for k in np.arange(0,lid):
                heatvalueindex[indexs[k]:indexe[k],i,j]=1
                if category[k]=='Moderate':
                    heatmoderate[indexs[k]:indexe[k],i,j]=1
                if category[k]=='Strong':
                    heatstrong[indexs[k]:indexe[k],i,j]=1
                if category[k]=='Severe':
                    heatsevere[indexs[k]:indexe[k],i,j]=1
                if category[k]=='Extreme':
                    heatextreme[indexs[k]:indexe[k],i,j]=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


x:/heat_content\marineHeatWave.py:388: RuntimeWarning: invalid value encountered in double_scalars
  mhw['rate_decline'].append((mhw_relSeas[tt_peak] - mhw_relSeas[-1]) / (tt_end-tt_start-tt_peak))


64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174


In [ ]:
plt.plot(newtime,xtest)

In [19]:
import pickle

In [ ]:
newtime.shape

In [20]:
pickle.dump([time,newtime],open("heat_wave_westcoast_time_sst20220809.p","wb"))

In [ ]:
#pickle.dump([heatwavesnumber,hwv,hwmoderate,hwstrong,hwsevere,hwextreme,hwevents,startindexs,stopindexs,heatvalueindex,heatmoderate,heatstrong,heatsevere], open("heat_gfnms.p","wb"))

In [21]:
pickle.dump([heatwavesnumber,hwv,hwmoderate,hwstrong,hwsevere,hwextreme,hwevents,startindexs,stopindexs,heatvalueindex], open("heat_1sst20220809.p","wb"))

In [24]:
pickle.dump([heatmoderate,heatstrong,heatsevere], open("heat_2sst20220809.p","wb"))

MemoryError: 

In [23]:
pickle.dump([heatextreme],open("heat_3sst20220809.p","wb"))

In [25]:
pickle.dump([heatmoderate],open("heat_moderate20220809.p","wb"))

In [26]:
pickle.dump([heatstrong],open("heat_strong20220809.p","wb"))

In [27]:
pickle.dump([heatsevere],open("heat_severe20220809.p","wb"))